# Parking Neural Network

---

In [1]:
import numpy as np
import pandas as pd

import keras.backend as K

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

from sklearn.preprocessing import StandardScaler

/home/ai/anaconda3/envs/keras_tensorflow_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ai/anaconda3/envs/keras_tensorflow_gpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Building data set

In [17]:
# Importing the dataset
dataset = pd.read_csv('../training_set.csv', sep=';')

In [18]:
dataset.head()

,NoTroncon,DateCreation,DayOfYear,DayOfWeek,NoHeure,TxOccupation,CenterLongitude,CenterLatitude,EmplacementCount,Temp,Precip_total_day_mm
0,1,2015-01-02,2,4,9,0.0000,-73.570339,45.507996,17,-11.0,0.1
1,1,2015-01-02,2,4,10,0.0000,-73.570339,45.507996,17,-11.0,0.1
2,1,2015-01-02,2,4,11,0.0000,-73.570339,45.507996,17,-11.0,0.1
3,1,2015-01-02,2,4,12,0.0510,-73.570339,45.507996,17,-10.0,0.1
4,1,2015-01-02,2,4,13,0.0588,-73.570339,45.507996,17,-12.0,0.1


In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153852 entries, 0 to 1153851
Data columns (total 11 columns):
NoTroncon              1153852 non-null int64
DateCreation           1153852 non-null object
DayOfYear              1153852 non-null int64
DayOfWeek              1153852 non-null int64
NoHeure                1153852 non-null int64
TxOccupation           1153852 non-null float64
CenterLongitude        1153852 non-null float64
CenterLatitude         1153852 non-null float64
EmplacementCount       1153852 non-null int64
Temp                   1153852 non-null float64
Precip_total_day_mm    1153852 non-null float64
dtypes: float64(5), int64(5), object(1)
memory usage: 96.8+ MB


## Building training matrix X and Y

In [20]:
Y_train = dataset['TxOccupation'].values
dataset.drop('TxOccupation', 1, inplace=True)

X_train = dataset.iloc[:, 2:8].values

In [21]:
print(X_train)

[[  2.         4.         9.       -73.570339  45.507996  17.      ]
 [  2.         4.        10.       -73.570339  45.507996  17.      ]
 [  2.         4.        11.       -73.570339  45.507996  17.      ]
 ...
 [366.         5.        15.       -73.569645  45.512401   2.      ]
 [366.         5.        16.       -73.569645  45.512401   2.      ]
 [366.         5.        17.       -73.569645  45.512401   2.      ]]


## Features scaling

In [22]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

In [23]:
print(X_train)

[[-1.70695075  0.72127126 -1.5835974  -1.40756526 -0.70498352  1.05272921]
 [-1.70695075  0.72127126 -1.28398011 -1.40756526 -0.70498352  1.05272921]
 [-1.70695075  0.72127126 -0.98436282 -1.40756526 -0.70498352  1.05272921]
 ...
 [ 1.71599998  1.26119268  0.21410635 -1.21645947  0.58374324 -1.27177442]
 [ 1.71599998  1.26119268  0.51372364 -1.21645947  0.58374324 -1.27177442]
 [ 1.71599998  1.26119268  0.81334093 -1.21645947  0.58374324 -1.27177442]]


In [24]:
print(sc.mean_)

[183.51884817   2.6641181   14.28540055 -73.56522743  45.51040571
  10.20674835]


In [25]:
print("Longitude mean = ", sc.mean_[3])
print("Latitude mean = ", sc.mean_[4])
print("Parking spot per section mean = ", sc.mean_[5])

Longitude mean =  -73.56522743172778
Latitude mean =  45.51040570584876
Parking spot per section mean =  10.206748352475014


In [26]:
print('Training matrix minimum value: {}', np.min(X_train))
print('Training matrix maximum value: {}', np.max(X_train))

Training matrix minimum value: {} -1.974401332184225
Training matrix maximum value: {} 4.307034294017197


# Making the Neural Network Model (ANN)

In [27]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 200, kernel_initializer = 'uniform', activation = 'relu', input_dim = 6))

classifier.add(Dense(units = 200, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 200, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [28]:
def threshold_accuracy(y_true, y_pred, threshold=0.10):
    return K.mean(K.abs(y_true - y_pred) < threshold, axis=-1)

In [29]:
# Compiling the ANN
adam = optimizers.Adam(lr=0.001)
classifier.compile(optimizer = adam, loss = 'mse', metrics=[threshold_accuracy])

In [30]:
classifier.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5_input (InputLayer)   (None, 6)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               1400      
_________________________________________________________________
dense_6 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_7 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 201       
Total params: 82,001
Trainable params: 82,001
Non-trainable params: 0
_________________________________________________________________


# Fitting the ANN to the training set

In [31]:
classifier.fit(X_train, Y_train, batch_size = 10000, epochs = 50)

Epoch 1/50
1153852/1153852 [==============================] - 1s - loss: 0.1127 - threshold_accuracy: 0.1674     
Epoch 2/50
1153852/1153852 [==============================] - 0s - loss: 0.0971 - threshold_accuracy: 0.2007     
Epoch 3/50
1153852/1153852 [==============================] - 0s - loss: 0.0908 - threshold_accuracy: 0.2160     
Epoch 4/50
1153852/1153852 [==============================] - 0s - loss: 0.0850 - threshold_accuracy: 0.2333     
Epoch 5/50
1153852/1153852 [==============================] - 0s - loss: 0.0813 - threshold_accuracy: 0.2528     
Epoch 6/50
1153852/1153852 [==============================] - 0s - loss: 0.0789 - threshold_accuracy: 0.2684     
Epoch 7/50
1153852/1153852 [==============================] - 0s - loss: 0.0773 - threshold_accuracy: 0.2779     
Epoch 8/50
1153852/1153852 [==============================] - 0s - loss: 0.0760 - threshold_accuracy: 0.2843     
Epoch 9/50
1153852/1153852 [==============================] - 0s - loss: 0.0754 - thresh